In [1]:
import torch
import torchmetrics
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import pytorch_lightning as L
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_lightning import Trainer
from scipy import stats
from torch.utils.data import DataLoader, TensorDataset, random_split

In [2]:
transform = transforms.Compose([transforms.ToTensor()])

train_dataset = datasets.MNIST(root="./data", train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root="./data", train=False, transform=transform, download=True)

def extrair_tensors(dataset):
    X = torch.stack([dataset[i][0] for i in range(len(dataset))])
    y = torch.tensor([dataset[i][1] for i in range(len(dataset))], dtype=torch.long)
    return X, y

X_treino, y_treino = extrair_tensors(train_dataset)
X_teste, y_teste = extrair_tensors(test_dataset)

tamanho_validacao = int(0.1 * len(X_treino))
tamanho_treino = len(X_treino) - tamanho_validacao

train_subset, val_subset = random_split(list(zip(X_treino, y_treino)), [tamanho_treino, tamanho_validacao])
X_treino, y_treino = extrair_tensors(train_subset)
X_validacao, y_validacao = extrair_tensors(val_subset)

In [3]:
class DataModule(L.LightningDataModule):
    def __init__(self, X_treino, y_treino, X_validacao, y_validacao, X_teste, y_teste, tamanho_lote=128, num_trabalhadores=4):
        super().__init__()
        self.X_treino = X_treino
        self.y_treino = y_treino
        self.X_validacao = X_validacao
        self.y_validacao = y_validacao
        self.X_teste = X_teste
        self.y_teste = y_teste
        self.tamanho_lote = tamanho_lote
        self.num_trabalhadores = num_trabalhadores

    def train_dataloader(self):
        return DataLoader(TensorDataset(self.X_treino, self.y_treino), batch_size=self.tamanho_lote, num_workers=self.num_trabalhadores)

    def val_dataloader(self):
        return DataLoader(TensorDataset(self.X_validacao, self.y_validacao), batch_size=self.tamanho_lote, num_workers=self.num_trabalhadores)

    def test_dataloader(self):
        return DataLoader(TensorDataset(self.X_teste, self.y_teste), batch_size=self.tamanho_lote, num_workers=self.num_trabalhadores)

In [4]:
class CNN(L.LightningModule):
    def __init__(self, num_filtros, num_neuronios_por_camada, taxa_aprendizado, dropout_rate, input_size=784, num_classes=10, act_function=nn.ReLU()):
        super().__init__()
        self.save_hyperparameters(ignore=['act_function'])
        
        self.camadas_conv = self._criar_camadas_convolucionais(num_filtros)
        
        self.fc = self._criar_camadas_fc(num_neuronios_por_camada, num_classes)
        
        self.fun_perda = nn.CrossEntropyLoss()

        self.acuracia = torchmetrics.Accuracy(num_classes=num_classes, task='multiclass')

    def _criar_camadas_convolucionais(self, num_filtros):
        camadas = []
        input_channels = 1
        for num_filtro in num_filtros:
            camadas.append(nn.Conv2d(input_channels, num_filtro, kernel_size=3, stride=1, padding=1))
            camadas.append(nn.ReLU())
            camadas.append(nn.MaxPool2d(kernel_size=2, stride=2))
            input_channels = num_filtro
        return nn.Sequential(*camadas)

    def _criar_camadas_fc(self, num_neuronios_por_camada, num_classes):
        tamanho_camada = 7 * 7 * self.hparams.num_filtros[-1] 
        
        camadas = []
        for n in num_neuronios_por_camada:
            camadas.append(nn.Linear(tamanho_camada, n))
            camadas.append(nn.ReLU())
            camadas.append(nn.Dropout(self.hparams.dropout_rate))
            tamanho_camada = n
        camadas.append(nn.Linear(tamanho_camada, num_classes))
        return nn.Sequential(*camadas)

    def forward(self, x):
        x = self.camadas_conv(x) 
        x = x.view(x.size(0), -1)
        return self.fc(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self(x)
        loss = self.fun_perda(y_pred, y)
        
        acuracia = self.acuracia(y_pred, y)
        
        self.log("train_loss", loss, prog_bar=True)
        self.log("train_acc", acuracia, prog_bar=True)
        
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self(x)
        loss = self.fun_perda(y_pred, y)
        
        acuracia = self.acuracia(y_pred, y)
        
        self.log("val_loss", loss, prog_bar=True)
        self.log("val_acc", acuracia, prog_bar=True)
        
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self(x)
        loss = self.fun_perda(y_pred, y)
        
        acuracia = self.acuracia(y_pred, y)
        
        self.log("test_loss", loss)
        self.log("test_acc", acuracia)
        
        return loss

    def on_epoch_end(self):
        self.log("train_acc_epoch", self.acuracia.compute())

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=self.hparams.taxa_aprendizado)

In [5]:
model = CNN(num_filtros=[16, 32], num_neuronios_por_camada=[28], taxa_aprendizado=0.001, dropout_rate=0.5)

dm = DataModule(X_treino, y_treino, X_validacao, y_validacao, X_teste, y_teste)


trainer = L.Trainer(
    max_epochs=500,
    callbacks=[EarlyStopping(monitor="val_loss", patience=50, mode="min")],
    enable_checkpointing=True,
    deterministic=True
)

trainer.fit(model, dm)

/home/maria23015/anaconda3/envs/pyscf/lib/python3.12/site-packages/torch/cuda/__init__.py:734: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/maria23015/anaconda3/envs/pyscf/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default

  | Name         | Type          

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [9]:
test_dataset = TensorDataset(X_teste, y_teste)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

trainer.test(model, dataloaders=test_loader)

SLURM auto-requeueing enabled. Setting signal handlers.


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.9894000291824341
        test_loss           0.05274214968085289
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.05274214968085289, 'test_acc': 0.9894000291824341}]